# TREC 2018 Precision Medicine

In [12]:
import json
from json2html import *
from IPython.display import HTML
import pandas

import os, sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
from trec_utils import utils, running, evaluation

In [13]:
config = utils.load_config()

In [14]:
topics_all = utils.get_topics('./topics/topics2017.xml')
qrels_all = utils.get_qrels('./gold-standard/trials.2017.qrels')
TOPIC = 2

In [15]:
topic = topics_all[(topics_all.topic==TOPIC)]
qrels = utils.qrels_of_topics(qrels_all, topic)
topic

,topic,disease,gene,gene1,gene2,gene3,sex,age,age_group
2,2,Colon cancer,"KRAS (G13D), BRAF (V600E)",KRAS (G13D),BRAF (V600E),,male,52,aged


In [16]:
run_params = {
    'run_id':'DEFAULT_RUN',
    'query_template':'tests/baseline-ct.json',
    'disease_tie_breaker':0.4,
    'disease_multi_match_type':'best_fields',
    'disease_boost':1.5,
    'gene_tie_breaker':0.5,
    'gene_multi_match_type':'cross_fields',
    'gene_boost':1
}

run, params = running.run(topic, 'TRIALS', run_params)

results, aggregated = evaluation.evaluate(qrels[(qrels.relev.isin([2]))], run)
run_this, params = running.run(topic, 'TRIALS', params)
aggregated

RUN: DEFAULT_RUN TOPICS: 1
RUN: DEFAULT_RUN TOPICS: 1


{'P_10': 0.0, 'Rprec': 0.0, 'ndcg': 0.348, 'recall_1000': 0.8235}

In [17]:
run, params = running.run(topic)
params['query_template'] = 'topic-optimized2.json'
results, aggregated = evaluation.evaluate(qrels[(qrels.relev.isin([2]))], run)
run, params = running.run(topic, params)
aggregated

TypeError: run() missing 2 required positional arguments: 'abstracts_or_trials' and 'run_params'

Check relevant documents missed and why (for topic 2 for example)

In [7]:
run_set = set(run[(run.TOPIC_NO==TOPIC)].ID)
qrels_set = set(qrels[(qrels.relev.isin([2]))].doc_id)
qrels_set.difference(run_set)

{'15735008',
 '16448675',
 '17293392',
 '17517901',
 '18097554',
 '18098337',
 '19959686',
 '21457162',
 '21681432',
 '22210186',
 '22972628',
 '23071293',
 '23112547',
 '23446022',
 '23462926',
 '23763264',
 '24025253',
 '24244575',
 '24522186',
 '24805134',
 '25073438',
 '25386108',
 '25469291',
 '25696788',
 '26359417',
 '26371285',
 '26623049',
 '27099668',
 'AACR_2014-2009',
 'AACR_2017-3066',
 'AACR_2017-4639',
 'AACR_2017-5162',
 'ASCO_32653-65'}

In [5]:
this_run_params = {
    'run_id':'ONE_RUN',
    'query_template':'variable/baseline_sex_age.json',
    'disease_tie_breaker':1,
    'disease_multi_match_type':'best_fields',
    'disease_boost':1,
    'gene_tie_breaker':0.2,
    'gene_multi_match_type':'best_fields',
    'gene_boost':2
}

full_run_df, run_params = running.run(topics_all, this_run_params)
results, aggregated = evaluation.evaluate(qrels_all, full_run_df)
aggregated
# Only MeSH, best fields: {'P_10': 0.5167, 'Rprec': 0.2785, 'ndcg': 0.5383, 'recall_1000': 0.6148}
# Only MeSH, cross fields: {'P_10': 0.5067, 'Rprec': 0.2811, 'ndcg': 0.5413, 'recall_1000': 0.6159}
# Only MeSH, cross, best: {'P_10': 0.52, 'Rprec': 0.2814, 'ndcg': 0.5378, 'recall_1000': 0.6151}
# Only MeSH, best, cross: {'P_10': 0.52, 'Rprec': 0.2787, 'ndcg': 0.5417, 'recall_1000': 0.6173} ***** best ndcg, best P_10

RUN: ONE_RUN TOPICS: 30


{'P_10': 0.4867, 'Rprec': 0.2629, 'ndcg': 0.5281, 'recall_1000': 0.6076}

In [5]:
run_params = running.get_default_run_params()
run_params['query_template'] = 'fixed/baseline.json'
full_run_df, run_params = running.run(topics_all)
results, aggregated = evaluation.evaluate(qrels_all, full_run_df)
aggregated
# Only MeSH, best fields: {'P_10': 0.5167, 'Rprec': 0.2785, 'ndcg': 0.5383, 'recall_1000': 0.6148}
# Only MeSH, cross fields: {'P_10': 0.5067, 'Rprec': 0.2811, 'ndcg': 0.5413, 'recall_1000': 0.6159}
# Only MeSH, cross, best: {'P_10': 0.52, 'Rprec': 0.2814, 'ndcg': 0.5378, 'recall_1000': 0.6151}
# Only MeSH, best, cross: {'P_10': 0.52, 'Rprec': 0.2787, 'ndcg': 0.5417, 'recall_1000': 0.6173} ***** best ndcg, best P_10

RUN: DEFAULT_RUN TOPICS: 30


{'P_10': 0.48, 'Rprec': 0.2847, 'ndcg': 0.5338, 'recall_1000': 0.6139}